In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

from scipy.stats import rv_histogram

import nice
from nice.reload import deep_reload

In [28]:
'''
Creating an atlas
'''

deep_reload(nice)

rng = np.random.default_rng(26893751)

kw = {
    'rng': rng,
    'xlim': (0, 1e6),
    'ylim': (0, 1e6),
    'create_using': nx.DiGraph
}

atlas = nice.graph.nmu_graph(100, 3, **kw)

for source, _adj in atlas._adj.items():
    for target, edge in _adj.items():

        edge['speed'] = rng.uniform(65, 65) * 1609 / 3600
        edge['time'] = edge['distance'] / edge['speed']
        edge['consumption'] = 500 * edge['distance']
        edge['cost'] = edge['time']

In [29]:
'''
Selecting places and stations
'''
deep_reload(nice)

n_places = 15
n_stations = 45

rng = np.random.default_rng(26589763514)

places = list(rng.choice(atlas.nodes, replace = False, size = n_places))
stations = list(rng.choice(
    list(set(atlas.nodes) - set(places)), replace = False, size = n_stations
))

In [31]:
'''
Transforming Graph
'''
deep_reload(nice)

conditions = [
    lambda e: e['consumption'] <= 80 * 3.6e6,
    lambda e: e['consumption'] >= 0 * 3.6e6,
]

kw = {
    'fields': ['time', 'distance', 'consumption']
}

nodes = places + stations

graph = nice.frp.transformed_graph(atlas, nodes, conditions, **kw)

In [7]:
paths = nice.frp.get_paths(
    graph, terminals = places, k = 10, weight = 'time',
)

In [26]:
deep_reload(nice)

energy = 35 * 3.6e6
power = 80e3
m = 1 / (energy / power)

queue = nice.queue.Queue(m = m)

rho = queue.max_rho(1)
rho, m

(array([0.08108948, 0.19631286, 0.28062976, 0.34264907, 0.39079445,
        0.42929044, 0.46098733, 0.48765621, 0.51075471, 0.53077168,
        0.54835063, 0.5639172 , 0.57817411, 0.59101442, 0.60251129,
        0.61313306, 0.62295592, 0.63209473, 0.64068829, 0.648461  ,
        0.65560509, 0.66255819, 0.66926382, 0.67504651, 0.68106311,
        0.68630631, 0.6916246 , 0.69640891, 0.70127895, 0.70551981,
        0.71020481, 0.7138651 , 0.71794952, 0.72169438, 0.72510154,
        0.72887459, 0.7320308 , 0.73508138, 0.73843235, 0.74138861,
        0.74402309, 0.74689314, 0.75001597, 0.75221787, 0.75459668,
        0.75717031, 0.75995825, 0.76188883, 0.76394369, 0.76615083,
        0.76852413, 0.77067367, 0.77237861, 0.77419645, 0.77613664,
        0.77820925, 0.78026076, 0.78170561, 0.7832362 , 0.78485902,
        0.78658098, 0.78840943, 0.7902115 , 0.79144439, 0.79274281,
        0.79411122, 0.79555435, 0.79707721, 0.79868508, 0.80022495,
        0.80127157, 0.8023678 , 0.80351668, 0.80

In [21]:
queue.waiting_times.shape, queue.c.shape

((100, 100), (100,))

In [181]:
'''
Adding classes
'''
deep_reload(nice)

rng = np.random.default_rng(26589763514)

for place in places:

    destinations = set(places) - set([place])
    demands = rng.uniform(0, 1, size = len(destinations))
    demands /= demands.sum()

    graph._node[place]['penalty'] = {}
    
    path = nx.shortest_path(graph, source = place, weight = 'ac_time')
    
    for destination in destinations:

        costs = nice.graph.path_cost(
            graph, path[destination], fields = ['time']
        )
        
        graph._node[place]['penalty'][destination] = costs['time'] * 200
    
    graph._node[place]['flows'] = {p: demands[i] for i, p in enumerate(destinations)}
    graph._node[place]['_class'] = nice.optimization.Place

energy = 35 * 3.6e6
power = 80e3
delay = 300
m = 1 / (energy / power)

queue = nice.queue.Queue(m = m)
c = queue.c
rho = queue.max_rho(delay)

for station in stations:

    graph._node[station]['power'] = 80e3
    graph._node[station]['_class'] = nice.optimization.Station

    graph._node[station]['volumes'] = rho * m * c
    graph._node[station]['delay'] = delay
    graph._node[station]['expenditures'] = c

for source, _adj in graph._adj.items():
    for target, edge in _adj.items():

        edge['_class'] = nice.optimization.Edge

for path in paths:

    path['_class'] = nice.optimization.Path